In [1]:
global phcloc = Cmd(`/Users/kv/Desktop/phc`) # executable

`/Users/kv/Desktop/phc`

In [2]:

"""
read_system(filename)

Input: (1) A filename containing text of polynomials separated by lines
       (2) An optional argument indicating the number of variables in the system
Output: A string array of polynomials.
"""
function read_system(filename; variables = nothing)
    count = 0;
    polyCount = 0;
    polynomial = [""]
    success_state = true 
    
    if (typeof(variables) != Int64 || variables < 1)
        variables = nothing
        println("Invalid variable type for optional argument")
    end
    ncolon = 0
    neq = 0
    nvar = 0
    for word in eachline(filename)
        word = lstrip(word)
        word = rstrip(word)
        if count == 0
            t  = split(word, r" *")
            neq = parse(Int,t[1])
            nvar = neq
            if length(t) == 2
                nvar = parse(Int,t[2])
            end
        # Note the first line      
        else
            ncolon = check_semicolon(word)
            if ncolon == 1
                push!(polynomial,word)
            else
                poly_list = split(word,";")
                for i = 1:length(poly_list)
                    word = rstrip(lstrip(poly_list[i]))
                    push!(polynomial,word)
                end
            end
        end
        count = count + 1
    end
    
    if ncolon != neq
        println("The number of colons does not match equations")
    end
        
    
    polynomial = setdiff(polynomial, [""])
    print(neq)
    print(" polynomials in the given system")
    println("")
    if (variables != nothing)
        print(variables)
        print(" variables in the given system")
        println("")
    end
    println(polynomial)
    return nvar,polynomial
end

read_system

In [3]:
function check_semicolon(string)
    ncolon = 0
    for i = 1:length(string)
        if string[i] == ";"
            ncolon = ncolon + 1
        end
    end
    return ncolon
end

check_semicolon (generic function with 1 method)

In [4]:
read_system("sample.txt")

Invalid variable type for optional argument
The number of colons does not match equations
2 polynomials in the given system
["x**2", "x-y"]


(2, ["x**2", "x-y"])

In [5]:
"""
write_system(filename,system)

Input: system - A polynomial system (array of strings of polynomials), filename - name of file to write the system

Output: The polynomial system written to a file.
"""
function write_system(filename,system)
    sys_nvar = size(system)[1]
    sys_npoly = sys_nvar
    f = open(filename, "w")
    s = "$sys_npoly"
    write(f,s)
    write(f,"\n")
    for i in 1:sys_nvar
        str = system[i]
        write(f,str)
        write(f,"\n")
    end
    close(f)
end

write_system

In [19]:
filename = "/Users/kv/Desktop/temp/in7769383274871036357"
    A = [1 0 0 0; 0 1 0 0;0 0 0 0;0 0 0 0;1 0 1 0;0 1 0 1;0 0 0 0;
              0 0 0 0;1 0 2 0;0 1 0 2; 0 0 0 0;0 0 0 0;1 0 3 0;0 1 0 3;0 0 0 0;0 0 0 0]

    coeff = [             1;
                1;
              9.98250904334731E-01;  
             0 ;                                              
              1;                                               
              1 ;                                              
              8.92749639148806E-01;  
              0;                                               
              1;                                               
              1;                                               
              1.60088552022675E-01;   
              0;                                               
              1;                                               
              1;                                               
              7.25369971319578E-01;  
             0]    
T = make_system(coeff,A)
print(T)
filename = "/Users/kv/Desktop/temp/sinha"
write_system(filename,T)

16["+x1+x2+0.998250904334", "+x1x3+x2x4+0.892749639148", "+x1x3**2+x2x4**2+0.160088552022", "+x1x3**3+x2x4**3+0.725369971319"]

In [20]:
    A = [1 0 0 0 0 0;0 1 0 0 0 0;0 0;1 0;0 1;0 0]

    coeff = [1;1;0;
            1;-1;0]
T = make_system(coeff,A)

LoadError: ArgumentError: argument count does not match specified shape (expected 36, got 20)

In [6]:
"""
form_poly(n_var,coeff,exponent)

Input: the number of variables, An array of coeffs for each polynomial, a 2D array of exponents
Preconditions - (1) Entry of coeff and corr. row of exponent matrices correspond to same term. 
                (2) exponent values are real, coefficients may be complex, given in format complex(a,b) or a+bim
                (3) The size of coeff must equal the number of rows of exponent.
Output: String representation of the polynomial
"""


function form_poly(n_var,coeff,exponent)
    variables = []
    terms = []
    polynomial = ""
    for i in 1:n_var
        b = string(i)
        str = "x" * b
        push!(variables, str)
    end
    
    row_exp = size(exponent)[1]
    col_exp = size(exponent)[2]
    n_coeffs = size(coeff)[1]
    
    
   # populates the terms list as strings
    for i in 1:col_exp
        term = "";
        for j in 1:row_exp
            if (exponent[j,i] != 0)
                if (exponent[j,i] == 1)
                    term = term * "*" * variables[j]
                else
                    term = term * "*" * variables[j]
                    term = term * "**" * string(exponent[j,i])
                end
            end
        end
        if (length(term) > 1 && term[1] == '*')
            term = term[2:end]
        end
        push!(terms,term)
    end
      print(terms)
    
    # Combines terms with coefficients and adds them to form the polynomial (sentinal case for 1st term)
    if (coeff[1] != 0 && coeff[1] != 1 && coeff[1] != -1)
        if imag(coeff[i] == 0)
            polynomial = polynomial * string(coeff[1]) * terms[1] 
        else
            polynomial = polynomial * '(' * string(coeff[1]) *')'* '*' * terms[1] 
        end
    end
    if (coeff[1] == 1)
        polynomial = polynomial * terms[1]   
    end
    
    # Appends "+" operators between terms
    if (n_coeffs > 1)
        for i in 2:n_coeffs-1
            if (coeff[i] != 0 && coeff[i] != 1 && coeff[i] != -1 && imag(coeff[i])==0)
                polynomial = polynomial * "+" * string(coeff[i]) * '*' * terms[i] 
            end
            if (imag(coeff[i]) != 0)
                polynomial = polynomial * "+" * '(' * string(coeff[i]) * ')' * '*' * terms[i] 
            end
            # Does not append a 1/-1 coefficient to terms, unless the term is "1"
            if (coeff[i] == 1 && terms[i] != "")
                polynomial = polynomial * "+" * terms[i]
            elseif (coeff[i] == -1 && terms[i] != "")
                polynomial = polynomial * "-" * terms[i]
            elseif (coeff[i] == 1 && terms[i] == " ")
                polynomial =  polynomial * ""
            elseif (coeff[i] == -1 && terms[i] == " ")
                polynomial =  polynomial * ""
            end
        print(terms)
        end
    end   
    return polynomial
end

form_poly (generic function with 1 method)

In [7]:
"""
make_system(tableau)

Takes an array representing a polynomial system in tableau format
Preconditions - (1) The system is assumed to be square
                (2) exponent values are real, coefficients may be complex, given in format complex(a,b) or a+bim
Arguments - (1) Coeff, contains a complex array of coefficients
            (2) tableau, contains an integer array of exponents associated with variables
            (3) variables, contains a string array of variables, otherwise variables default to x1,x2,....

Domain Exceptions - (1) Length of coeff does not match the number of rows of the tableau
                    (2) The optional variable array is not of type String
                    (3) The optional variable array size does not match the col size of tableau

Outputs a string array of polynomials as represented by the tableau system
"""


# im - should be converted to i
# + - (coefficients are negative do not insert + -)
# Trailing brackets issue

function make_system(coeff, tableau; variables = nothing)::Array{String}
    n_row = size(tableau)[1]
    n_col = size(tableau)[2]
    p_array = String[]
    variables = String[]
    n_poly = 0
    n_var = n_col;
    
    if (n_row != length(coeff))
        println("Coeff length does not equal the number of tableau rows!")
        throw(DomainError())
    end
    
    if (length(variables) > 0 && typeof(variables) != String)
        println("Variable array must contain strings only")
        throw(DomainError())
    end     
    
    
    if (length(variables) > 0 && length(variables) != n_var)
        println("Variable array dimensions do not match tableau column size")
        throw(DomainError())
    end 
    
    # Count the number of polynomials in the tableau.
    for i in 1:n_row
        if tableau[i,1] == 0
            n_poly = n_poly + 1
        end
    end
    
    
    
    # If optional variables are not supplied
    if (length(variables) == 0)
        for i in 1:n_var
            b = string(i)
            str = "x" * b
            push!(variables, str)
            str = ""
        end
    end
    
    poly = ""
    print(n_row)
        
    for i in 1:n_row

        if (coeff[i] != 0)
            sum = 0 # tracks whether a term has variables or not 
            for j in 1:n_var
                # append coefficient here if nonzero or non-unity
                if (j == 1)
                    poly = poly * "+"
                    if (coeff[i] != 0 && coeff[i] != 1)
                        if (imag(coeff[i]) == 0)
                            if (real(coeff[i]) > 0)
                                poly = poly * string(real(coeff[i]))
                            else
                                poly = poly * "(" * string(real(coeff[i])) * ")"
                            end 
                            
                        else
                            poly = poly * "(" * string((coeff[i])) * ")"
                            replace(poly, "im" => "I", count = 1)
                        end
                    end
                end
                
                
                
                # append variables and exponents
                if (tableau[i,j] == 1)
                    if (coeff[i] != 1)
                        poly = poly * "*" * variables[j]
                    else
                        poly = poly * variables[j]
                    end
                elseif (tableau[i,j] > 1)
                    if (coeff[i] != 1)
                        poly = poly * "*" * variables[j] * "**" * string(tableau[i,j])
                    else
                        poly = poly * variables[j] * "**" * string(tableau[i,j])
                    end
                end
                
                
                sum = sum + tableau[i,j] #nonzero sum means has variables
            end
            """
            if (i < n_row)
                if (length(poly) > 0 && poly[end] != " " && poly[1] != "-")
                        poly = poly[1:end-1] * "+"
                end
            else
                poly = poly[1:end-1]
            end
            """
                
            
            #append final coefficient
            
            if (sum == 0)
                poly = poly[1:end-3];
            end
        else
            push!(p_array,poly)
            poly = ""
        end     
    end
    
    return p_array
end
        
    
    

make_system (generic function with 1 method)

In [8]:
"""
extract_sols(char_sols)

Take a string representation of the solution system
Preconditions - (1) The solutions are separated by "[]", as formatted by phc
                (2) Within solutions, attributes are separated by ","
Arguments - (1) char_sols - string output representation of solution system

Outputs a list of dictionaries that contain the solution systems
"""
function extract_sols(char_sols)
    sol_list = split(char_sols ,r"[[]")
    deleteat!(sol_list, 1)
    dict_list = []
    print(length(sol_list))
    for i in 1:length(sol_list)
        dict = Dict()
        attr_list = split(sol_list[i],r"[,]")
        deleteat!(attr_list, length(attr_list)) # deletes last element
        for j in 1:length(attr_list)
            indiv_list = split(attr_list[j],r"[=]")
            indiv_list[1] = replace(indiv_list[1],"\n"=>"")
            indiv_list[1] = lstrip(indiv_list[1])
            indiv_list[1] = rstrip(indiv_list[1])
            if (length(indiv_list) > 1)
                indiv_list[2] = replace(indiv_list[2],"\n"=>"")
                indiv_list[2] = lstrip(indiv_list[2])
                indiv_list[2] = rstrip(indiv_list[2])
                cpx_num = replace(indiv_list[2],"*I"=>"im")
                cpx_result = strToCpx(cpx_num)
                push!(dict, indiv_list[1] => cpx_result)
            end
            
            
        end
    sort(collect(dict), by=x->x[1])
    push!(dict_list, dict)
    end
return dict_list
end

"""
strToCpx(str)
Take a string and converts it to Complex format
Input: String of the form a+bI,a-bI
Outputs a complex double format, unless the string does not contain an imaginary component
"""
#Coverts a string to a complex double
function strToCpx(str)
    if (occursin("]",str) && length(str) > 1)
        str = str[1:end-1]
    end
    if (occursin("im", str))
        cpx = parse(Complex{Float64}, str)
    else
        cpx = parse(Float64,str)
    end
    return cpx
end
    

strToCpx (generic function with 1 method)

In [42]:
char_sols = "[time = 1.0 + 0*I,
  multiplicity = 1,
  x1 = 1.03639570141331 + 6.19038489119192E-3*I,
  x2 = -3.81447970785749E-2 - 6.53100262542169E-2*I,
  x3 = 7.26646740724671E-1 - 5.51989879497413E-1*I,
  x4 = -2.24458038436995 + 7.75072227352325E-1*I,
  err =  4.808E-16,  rco =  8.492E-03,  res =  5.837E-16],
 [time = 1.0 + 0*I,
  multiplicity = 1,
  x1 = -3.81447970785749E-2 - 6.53100262542169E-2*I,
  x2 = 1.03639570141331 + 6.19038489119191E-3*I,
  x3 = -2.24458038436995 + 7.75072227352325E-1*I,
  x4 = 7.26646740724671E-1 - 5.51989879497413E-1*I,
  err =  3.705E-16,  rco =  9.643E-03,  res =  6.800E-16]";
extract_sols(char_sols)

2

2-element Array{Any,1}:
 Dict{Any,Any}("err" => 4.808e-16,"x1" => 1.03639570141331 + 0.00619038489119192im,"time" => 1.0 + 0.0im,"x4" => -2.24458038436995 + 0.775072227352325im,"rco" => 0.008492,"multiplicity" => 1.0,"x2" => -0.0381447970785749 - 0.0653100262542169im,"res" => 5.837e-16,"x3" => 0.726646740724671 - 0.551989879497413im)
 Dict{Any,Any}("err" => 3.705e-16,"x1" => -0.0381447970785749 - 0.0653100262542169im,"time" => 1.0 + 0.0im,"x4" => 0.726646740724671 - 0.551989879497413im,"rco" => 0.009643,"multiplicity" => 1.0,"x2" => 1.03639570141331 + 0.00619038489119191im,"x3" => -2.24458038436995 + 0.775072227352325im)

## x = parse(Float64,"5")
print(x)

In [23]:
function main()
    # Test for Read/Write
    #read_system("Ex1.txt",variables = "fi")
    #sys = read_system("Ex1.txt")
    #print(sys[1])
    #write_system("Ex1-ou.txt",sys)
    
    
    # Test cases for form_poly(n_var,coeff,exponent)
    exp = [1 0 2 0;0 1 2 0;2 0 0 0;0 2 0 0]
    coeff = [1;1;complex(1,1);1]
    #size(exp)[1]
    poly = form_poly(4,coeff,exp)
    #print(poly)
    
    # Test for Make System
    A = [1 0 0 0; 0 1 0 0;0 0 0 0;0 0 0 0;1 0 1 0;0 1 0 1;0 0 0 0;
              0 0 0 0;1 0 2 0;0 1 0 2; 0 0 0 0;0 0 0 0;1 0 3 0;0 1 0 3;0 0 0 0;0 0 0 0]

    coeff = [             2;
                1+ im;
              -9.98250904334731E-01 + 5.91196413630250E-02*im;  
             0 ;                                              
              1;                                               
              1 ;                                              
              -8.92749639148806E-01 + 4.50553084330444E-01*im;  
              0;                                               
              1;                                               
              1;                                               
              1.60088552022675E-01 + 9.87102657027770E-01*im;   
              0;                                               
              1;                                               
              1;                                               
              -7.25369971319578E-01 + 6.88359211972815E-01*im;  
             0]      
    
    make_system(coeff,A)
    
    
    
char_sols = "[[time = 1.0 + 0*I,
  multiplicity = 1,
  x1 = 1.20279913210395E-1 + 3.65755965210328E-99*I,
  x2 = -1.11853081754512 - 5.48633947815492E-99*I,
  x3 = -1.544189366652 + 2.19453579126197E-98*I,
  x4 = 6.32092263402443E-1 - 3.65755965210328E-99*I,
  err =  1.324E-16,  rco =  3.725E-02,  res =  2.498E-16],
 [time = 1.0 + 0*I,
  multiplicity = 1,
  x1 = -1.11853081754512 - 2.28597478256455E-99*I,
  x2 = 1.20279913210395E-1 + 1.82877982605164E-99*I,
  x3 = 6.32092263402443E-1 - 7.14367119551422E-100*I,
  x4 = -1.544189366652 + 7.31511930420656E-99*I,
  err =  1.264E-16,  rco =  3.725E-02,  res =  1.665E-16]]"
    extract_sols(char_sols)

end
main()

Any["x1*x3**2", "x2*x4**2", "x1**2*x2**2", ""]Any["x1*x3**2", "x2*x4**2", "x1**2*x2**2", ""]Any["x1*x3**2", "x2*x4**2", "x1**2*x2**2", ""]163

3-element Array{Any,1}:
 Dict{Any,Any}()
 Dict{Any,Any}("err" => 1.324e-16,"x1" => 0.120279913210395 + 3.65755965210328e-99im,"time" => 1.0 + 0.0im,"x4" => 0.632092263402443 - 3.65755965210328e-99im,"rco" => 0.03725,"multiplicity" => 1.0,"x2" => -1.11853081754512 - 5.48633947815492e-99im,"res" => 2.498e-16,"x3" => -1.544189366652 + 2.19453579126197e-98im)
 Dict{Any,Any}("err" => 1.264e-16,"x1" => -1.11853081754512 - 2.28597478256455e-99im,"time" => 1.0 + 0.0im,"x4" => -1.544189366652 + 7.31511930420656e-99im,"rco" => 0.03725,"multiplicity" => 1.0,"x2" => 0.120279913210395 + 1.82877982605164e-99im,"x3" => 0.632092263402443 - 7.14367119551422e-100im)

In [9]:
function set_phcpath(pathstr = nothing)
    global phctemp
    phcloc = "/Users/kv/Desktop/phc"
    phctemp = "/tmp/"
    
    #mkdir(phctemp)

end


set_phcpath (generic function with 2 methods)

In [10]:
using Dates
using Random

function solve_system(T)

    global phcloc = "/Users/kv/Desktop/phc"#location of the phc executable
    global phctemp
    
    # Uses time to generate a random string
    moment = Dates.now()
    seed = Dates.value(moment)
    rng = MersenneTwister(seed)
    sr = string(abs(round(rand(rng, Int64)))) 
    print(sr)
    verbose=false;
    
    if(verbose) 
         print("solving polynomial system ... please wait...\n")
    end
    infile = phctemp * "in" * sr
    outfile = phctemp * "out" * sr
    solfile = phctemp * "sol" * sr
    # write system
    #print(solfile)
    write_system(infile, T)
    #close(infile)
    # call black box solver
    cmd_b = `$phcloc -b $infile $outfile`
    cmd_z = `$phcloc -z $infile $solfile`
    run(cmd_b)
    run(cmd_z)
    # read solutions in
    solf_id = open(solfile, "r")
    print(solfile)
    sols = string(read(solf_id))
    close(solf_id)
    #char_sols = char(sols) # 1*m string array
    # extract individual solution
    solutions = extract_sols(sols)
    
    # print out solutions info.
    if(verbose) 
        print("Blackbox solver found %d solutions.\n", length(solutions,2))
        for k=1:length(solutions,2)
           print(solutions(k))
        end
    end
    
    # remove all intermediate files
  
end

solve_system (generic function with 1 method)

In [16]:

S = ["x1**2;","x2*x2;"]
set_phcpath("/Users/kv/Desktop/phc")
solve_system(S)

/tmp/sol63983541470085306381

In [33]:
infile = Cmd(`/Users/kv/Desktop/temp/in2288155511451396793`)
outfile = Cmd(`/Users/kv/Desktop/temp/2288155511451396793`)
phcloc = Cmd(`/Users/kv/Desktop/phc`)
str = `$phcloc -b $infile $outfile`
run(str)


The file /Users/kv/Desktop/temp/in2288155511451396793 could not be found, please try again...
Give a string of characters : Something is wrong with argument file...
Welcome to PHC (Polynomial Homotopy Continuation) v2.4.78 30 Jun 2020.
Running the blackbox solver, no tasking, in double precision.

Is the system on a file ? (y/n/i=info) Oops, an unhandled exception occurred.
Seed used in random number generators : 51777.
Use the seed number to reproduce the error.



raised ADA.IO_EXCEPTIONS.END_ERROR : a-textio.adb:517


LoadError: failed process: Process(`/Users/kv/Desktop/phc -b /Users/kv/Desktop/temp/in2288155511451396793 /Users/kv/Desktop/temp/2288155511451396793`, ProcessExited(1)) [1]


In [30]:
run(`/Users/kv/Desktop/phc`)

Welcome to PHC (Polynomial Homotopy Continuation) v2.4.78 30 Jun 2020

Running full mode, no tasking.  Note also the following options:
  phc -0 : random numbers with fixed seed for repeatable runs    
  phc -a : solving polynomial systems equation-by-equation       
  phc -b : batch or black-box processing, the blackbox solver    
  phc -B : blackbox numerical irreducible decomposition solver   
  phc -c : irreducible decomposition for solution components     
  phc -d : linear and nonlinear reduction w.r.t. the total degree
  phc -e : SAGBI/Pieri/Littlewood-Richardson homotopies          
  phc -f : factor pure dimensional solution set into irreducibles
  phc -g : checking whether an input system has the right syntax 
  phc -h : displays help, e.g.: phc -h -b or phc -b -h, or --help
  phc -j : path tracking with algorithmic differentiation        
  phc -k : realization of dynamic output feedback placing poles  
  phc -l : witness set for hypersurface cutting with random line 
  phc 


raised ADA.IO_EXCEPTIONS.END_ERROR : a-tigeli.adb:96


LoadError: failed process: Process(`/Users/kv/Desktop/phc`, ProcessExited(1)) [1]


In [60]:
Q = split("x",r"[=]")
Q[2]

LoadError: BoundsError: attempt to access 1-element Array{SubString{String},1} at index [2]

In [80]:
sol_list = split("[a,b,c]" ,r"[[]")
length(sol_list)

2

In [58]:
size([1,2])[1]

2

In [23]:
string = "5   2"
t  = split(string, r" *")
neq = parse(Int,t[1])
nvar = neq
if length(t) == 2
    nvar = parse(Int,t[2])
end
    
print(neq)
print("\n")
print(nvar)

5
2

In [35]:
string = "3 2 ;"
string[end]

';': ASCII/Unicode U+003B (category Po: Punctuation, other)

In [89]:
q = split("Geeks, for, Geeks", ", ")

3-element Array{SubString{String},1}:
 "Geeks"
 "for"
 "Geeks"

In [108]:
word = "5"

t  = split("5", r" *")

1-element Array{SubString{String},1}:
 "5"